# AI Config Custom Metrics - Cookbook

Full lifecycle: Create, Track, Get, Update, Delete metrics.

Prerequisites: `LAUNCHDARKLY_SDK_KEY`, `LAUNCHDARKLY_API_TOKEN`

In [1]:
%pip install launchdarkly-server-sdk python-dotenv requests -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

repo_root = find_repo_root()
load_dotenv(repo_root / '.env')
print(f"[OK] Loaded environment from {repo_root / '.env'}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment from /Users/ld_scarlett/Documents/Github/agent-skills/.env


In [2]:
# SDK initialization (see aiconfig-sdk for details)
from ldclient import Context
from ldclient.config import Config
import ldclient
import requests

SDK_KEY = os.environ.get("LAUNCHDARKLY_SDK_KEY")
API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
PROJECT_KEY = "support-ai"

ldclient.set_config(Config(SDK_KEY))
ld_client = ldclient.get()
print(f"[OK] SDK initialized: {ld_client.is_initialized()}")
print(f"[OK] API token configured: {bool(API_TOKEN)}")

[OK] SDK initialized: True
[OK] API token configured: True


---
## 1. Create Metric (API)
From: `SKILL.md` lines 32-78

In [3]:
def create_metric(
    project_key: str,
    metric_key: str,
    name: str,
    kind: str = "custom",
    is_numeric: bool = True,
    unit: str = "count",
    success_criteria: str = "HigherThanBaseline",
    event_key: str = None,
    description: str = None
):
    """Create a new metric definition in LaunchDarkly."""
    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")

    url = f"https://app.launchdarkly.com/api/v2/metrics/{project_key}"

    payload = {
        "key": metric_key,
        "name": name,
        "kind": kind,
        "isNumeric": is_numeric,
        "eventKey": event_key or metric_key
    }

    # Unit and successCriteria are required for numeric custom metrics
    if is_numeric and kind == "custom":
        payload["unit"] = unit
        payload["successCriteria"] = success_criteria

    if description:
        payload["description"] = description

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 201:
        print(f"[OK] Created metric: {metric_key}")
        return response.json()
    elif response.status_code == 409:
        print(f"[INFO] Metric already exists: {metric_key}")
        return None
    else:
        print(f"[ERROR] Failed to create metric: {response.status_code}")
        print(f"        {response.text}")
        return None

print("[OK] create_metric() defined")

[OK] create_metric() defined


In [4]:
# Test create_metric
print("=== Testing create_metric ===")
TEST_METRIC_KEY = "cookbook.test.metric"

result = create_metric(
    PROJECT_KEY,
    TEST_METRIC_KEY,
    name="Cookbook Test Metric",
    kind="custom",
    is_numeric=True,
    description="Test metric created by cookbook"
)

=== Testing create_metric ===
[OK] Created metric: cookbook.test.metric


---
## 2. Track Events (SDK)
From: `SKILL.md` lines 89-165

In [5]:
def track_metric(ld_client, user_id: str, metric_key: str, value: float, data: dict = None):
    """Track an event to a metric."""
    context = Context.builder(user_id).build()

    ld_client.track(
        metric_key,
        context,
        data=data,
        metric_value=value
    )

print("[OK] track_metric() defined")

[OK] track_metric() defined


In [6]:
# Test track_metric
print("=== Testing track_metric ===")
track_metric(ld_client, "cookbook-user", TEST_METRIC_KEY, 100.0)
print(f"[OK] Tracked: {TEST_METRIC_KEY} = 100.0")

track_metric(ld_client, "cookbook-user", TEST_METRIC_KEY, 85.0, {"source": "test"})
print(f"[OK] Tracked: {TEST_METRIC_KEY} = 85.0 with metadata")

ld_client.flush()
print("[OK] Events flushed")

=== Testing track_metric ===
[OK] Tracked: cookbook.test.metric = 100.0
[OK] Tracked: cookbook.test.metric = 85.0 with metadata
[OK] Events flushed


---
## Common Tracking Patterns
From: `SKILL.md` lines 112-165

In [7]:
def track_conversion(ld_client, user_id: str, amount: float, config_key: str):
    """Track a conversion event with revenue."""
    context = Context.builder(user_id).build()

    ld_client.track(
        "business.conversion",
        context,
        data={"configKey": config_key, "category": "electronics"},
        metric_value=amount
    )

def track_task_success(ld_client, user_id: str, task_type: str, success: bool):
    """Track task completion success/failure."""
    context = Context.builder(user_id).build()

    ld_client.track(
        "task.success_rate",
        context,
        data={"taskType": task_type},
        metric_value=1.0 if success else 0.0
    )

def track_satisfaction(ld_client, user_id: str, score: float, feedback_type: str):
    """Track user satisfaction (0-100 scale)."""
    context = Context.builder(user_id).build()

    ld_client.track(
        "user.satisfaction",
        context,
        data={"feedbackType": feedback_type},
        metric_value=score
    )

    # Track negative feedback separately for alerts
    if score < 50:
        ld_client.track(
            "user.negative_feedback",
            context,
            metric_value=1.0
        )

def track_revenue(ld_client, user_id: str, revenue: float, source: str):
    """Track revenue generated after AI interaction."""
    context = Context.builder(user_id).set("tier", "premium").build()

    if revenue > 0:
        ld_client.track(
            "revenue.impact",
            context,
            data={"source": source},
            metric_value=revenue
        )

print("[OK] Common tracking patterns defined")

[OK] Common tracking patterns defined


In [8]:
# Test common tracking patterns
print("=== Testing Common Patterns ===")

track_conversion(ld_client, "cookbook-user", 150.00, "content-assistant")
print("[OK] Tracked: business.conversion = 150.00")

track_task_success(ld_client, "cookbook-user", "summarization", True)
print("[OK] Tracked: task.success_rate = 1.0")

track_satisfaction(ld_client, "cookbook-user", 85.0, "thumbs_up")
print("[OK] Tracked: user.satisfaction = 85.0")

track_revenue(ld_client, "cookbook-user", 99.99, "recommendation")
print("[OK] Tracked: revenue.impact = 99.99")

ld_client.flush()
print("[OK] Events flushed")

=== Testing Common Patterns ===
[OK] Tracked: business.conversion = 150.00
[OK] Tracked: task.success_rate = 1.0
[OK] Tracked: user.satisfaction = 85.0
[OK] Tracked: revenue.impact = 99.99
[OK] Events flushed


---
## 3. Get Metrics (API)
From: `SKILL.md` lines 171-223

In [9]:
def get_metric(project_key: str, metric_key: str):
    """Get a single metric definition."""
    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")

    url = f"https://app.launchdarkly.com/api/v2/metrics/{project_key}/{metric_key}"

    headers = {"Authorization": API_TOKEN}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        metric = response.json()
        print(f"[OK] Metric: {metric['key']}")
        print(f"     Name: {metric.get('name', 'N/A')}")
        print(f"     Kind: {metric.get('kind', 'N/A')}")
        print(f"     Numeric: {metric.get('isNumeric', False)}")
        print(f"     Event Key: {metric.get('eventKey', 'N/A')}")
        return metric
    elif response.status_code == 404:
        print(f"[INFO] Metric not found: {metric_key}")
        return None
    else:
        print(f"[ERROR] Failed to get metric: {response.status_code}")
        return None

print("[OK] get_metric() defined")

[OK] get_metric() defined


In [10]:
# Test get_metric
print("=== Testing get_metric ===")
metric = get_metric(PROJECT_KEY, TEST_METRIC_KEY)

=== Testing get_metric ===
[OK] Metric: cookbook.test.metric
     Name: Cookbook Test Metric
     Kind: custom
     Numeric: True
     Event Key: cookbook.test.metric


In [11]:
def list_metrics(project_key: str, limit: int = 20):
    """List all metrics in a project."""
    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")

    url = f"https://app.launchdarkly.com/api/v2/metrics/{project_key}"

    headers = {"Authorization": API_TOKEN}
    params = {"limit": limit}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        metrics = data.get("items", [])
        print(f"[OK] Found {len(metrics)} metrics:")
        for metric in metrics:
            numeric = "numeric" if metric.get("isNumeric") else "non-numeric"
            print(f"     - {metric['key']} ({metric.get('kind', 'custom')}, {numeric})")
        return metrics
    else:
        print(f"[ERROR] Failed to list metrics: {response.status_code}")
        return None

print("[OK] list_metrics() defined")

[OK] list_metrics() defined


In [12]:
# Test list_metrics
print("=== Testing list_metrics ===")
metrics = list_metrics(PROJECT_KEY, limit=10)

=== Testing list_metrics ===
[OK] Found 10 metrics:
     - ld_autogen__ai-input-tokens (custom, numeric)
     - ld_autogen__ai-output-tokens (custom, numeric)
     - ld_autogen__ai-total-tokens (custom, numeric)
     - ld_autogen__ai-completion-duration (custom, numeric)
     - ld_autogen__ai-time-to-first-token (custom, numeric)
     - ld_autogen__ai-positive-feedback-count (custom, non-numeric)
     - ld_autogen__ai-positive-feedback-rate (custom, non-numeric)
     - ld_autogen__ai-negative-feedback-count (custom, non-numeric)
     - ld_autogen__ai-negative-feedback-rate (custom, non-numeric)
     - ld_autogen__ai-completion-success (custom, non-numeric)


---
## 4. Update Metric (API)
From: `SKILL.md` lines 227-268

In [13]:
def update_metric(project_key: str, metric_key: str, updates: list):
    """
    Update a metric using JSON Patch operations.

    Args:
        updates: List of patch operations, e.g.:
            [{"op": "replace", "path": "/name", "value": "New Name"}]
    """
    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")

    url = f"https://app.launchdarkly.com/api/v2/metrics/{project_key}/{metric_key}"

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json"
    }

    response = requests.patch(url, json=updates, headers=headers)

    if response.status_code == 200:
        print(f"[OK] Updated metric: {metric_key}")
        return response.json()
    elif response.status_code == 404:
        print(f"[ERROR] Metric not found: {metric_key}")
        return None
    else:
        print(f"[ERROR] Failed to update metric: {response.status_code}")
        print(f"        {response.text}")
        return None

def rename_metric(project_key: str, metric_key: str, new_name: str, new_description: str = None):
    """Rename a metric and optionally update description."""
    updates = [
        {"op": "replace", "path": "/name", "value": new_name}
    ]
    if new_description:
        updates.append({"op": "replace", "path": "/description", "value": new_description})

    return update_metric(project_key, metric_key, updates)

print("[OK] update_metric() and rename_metric() defined")

[OK] update_metric() and rename_metric() defined


In [14]:
# Test update_metric
print("=== Testing update_metric ===")
result = rename_metric(
    PROJECT_KEY,
    TEST_METRIC_KEY,
    "Cookbook Test Metric (Updated)",
    "Updated description from cookbook"
)

# Verify the update
if result:
    print(f"[OK] New name: {result.get('name')}")

=== Testing update_metric ===
[OK] Updated metric: cookbook.test.metric
[OK] New name: Cookbook Test Metric (Updated)


---
## 5. Delete Metric (API)
From: `SKILL.md` lines 272-292

In [15]:
def delete_metric(project_key: str, metric_key: str):
    """Delete a metric from the project."""
    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")

    url = f"https://app.launchdarkly.com/api/v2/metrics/{project_key}/{metric_key}"

    headers = {"Authorization": API_TOKEN}

    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"[OK] Deleted metric: {metric_key}")
        return True
    elif response.status_code == 404:
        print(f"[INFO] Metric not found: {metric_key}")
        return False
    else:
        print(f"[ERROR] Failed to delete metric: {response.status_code}")
        return False

print("[OK] delete_metric() defined")

[OK] delete_metric() defined


In [16]:
# Test delete_metric - cleanup the test metric
print("=== Testing delete_metric ===")
delete_metric(PROJECT_KEY, TEST_METRIC_KEY)

# Verify deletion
print("\n=== Verifying deletion ===")
get_metric(PROJECT_KEY, TEST_METRIC_KEY)

=== Testing delete_metric ===
[OK] Deleted metric: cookbook.test.metric

=== Verifying deletion ===
[INFO] Metric not found: cookbook.test.metric


---
## Session Metrics Tracker
From: `SKILL.md` lines 343-403

In [17]:
import time

class SessionMetricsTracker:
    """Track metrics across an entire user session."""

    def __init__(self, ld_client):
        self.ld_client = ld_client
        self.session_data = {}

    def start_session(self, user_id: str, session_id: str):
        """Initialize session tracking."""
        self.session_data[session_id] = {
            "user_id": user_id,
            "start_time": time.time(),
            "interactions": 0,
            "successful_tasks": 0
        }

    def track_interaction(self, session_id: str, success: bool):
        """Track individual interaction within session."""
        if session_id not in self.session_data:
            return
        session = self.session_data[session_id]
        session["interactions"] += 1
        if success:
            session["successful_tasks"] += 1

    def end_session(self, session_id: str):
        """Finalize and track session metrics."""
        if session_id not in self.session_data:
            return None

        session = self.session_data[session_id]
        duration = time.time() - session["start_time"]

        context = Context.builder(session["user_id"]).build()

        # Track session duration
        self.ld_client.track(
            "session.duration",
            context,
            data={"interactions": session["interactions"]},
            metric_value=duration
        )

        # Track session success rate
        if session["interactions"] > 0:
            success_rate = session["successful_tasks"] / session["interactions"]
            self.ld_client.track(
                "session.success_rate",
                context,
                metric_value=success_rate * 100
            )

        result = dict(session)
        result["duration"] = duration
        del self.session_data[session_id]
        return result

print("[OK] SessionMetricsTracker class defined")

[OK] SessionMetricsTracker class defined


In [18]:
# Test SessionMetricsTracker
print("=== Testing SessionMetricsTracker ===")
tracker = SessionMetricsTracker(ld_client)
tracker.start_session("cookbook-user", "test-session-001")
tracker.track_interaction("test-session-001", success=True)
tracker.track_interaction("test-session-001", success=True)
tracker.track_interaction("test-session-001", success=False)
result = tracker.end_session("test-session-001")
print(f"[OK] Session: {result['interactions']} interactions, {result['successful_tasks']} successful")
print(f"[OK] Success rate: {(result['successful_tasks']/result['interactions'])*100:.1f}%")

=== Testing SessionMetricsTracker ===
[OK] Session: 3 interactions, 2 successful
[OK] Success rate: 66.7%


In [19]:
# Final flush
ld_client.flush()
print("[OK] All events flushed")

[OK] All events flushed
